In [ ]:
# Instalar Selenium, BeautifulSoup, Pandas y Webdriver Manager
!pip install selenium beautifulsoup4 pandas webdriver-manager

# --- INSTALACIÓN DE GOOGLE CHROME EN COLAB ---
# 1. Descargar la clave GPG de Google Chrome
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo gpg --dearmor -o /usr/share/keyrings/google-chrome-archive-keyring.gpg

# 2. Añadir el repositorio de Google Chrome a las fuentes de apt
!echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list

# 3. Actualizar los listados de paquetes apt para incluir el nuevo repositorio
!sudo apt-get update

# 4. Instalar Google Chrome estable
!sudo apt-get install -y google-chrome-stable

# Opcional: Verificar la versión de Chrome instalada
!google-chrome --version

print("\n--- Instalación de Chrome y dependencias completada ---")

gpg: cannot open '/dev/tty': No such device or address
deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://dl.google.com/linux/chrome/deb stable InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 384 k

In [ ]:
# --- PASO 1: INSTALAR LAS LIBRERÍAS NECESARIAS EN GOOGLE COLAB ---
# ¡IMPORTANTE!: Ejecuta esta celda al inicio de tu notebook.
# Esto intentará suprimir la mayoría de los mensajes de instalación.
!pip install selenium webdriver-manager > /dev/null 2>&1

# --- PASO 2: IMPORTAR LAS LIBRERÍAS ---
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re

# Importar para mostrar HTML en Colab
from IPython.display import display, HTML

def extraer_planes_bitel_colab():
    """
    Extrae la información de los planes postpago de Bitel Perú desde su página web.
    Usa Selenium para manejar el contenido dinámico y BeautifulSoup para parsear el HTML.
    Retorna una lista de diccionarios, con los planes extraídos.
    """
    url = "https://bitel.com.pe/planes/control/ilimitado"
    planes_data = []

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1990,1080') # Aumentado ligeramente el tamaño de la ventana
    options.add_argument('--log-level=3') # Suprime la mayoría de los logs del navegador
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    driver = None
    wait = None # Inicializar 'wait' aquí para asegurar que siempre esté definido

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        wait = WebDriverWait(driver, 45) # Ahora 'wait' se define aquí, antes de cualquier uso

        # --- MANEJO DE POP-UP DE COOKIES O SIMILAR ---
        try:
            common_close_xpaths = [
                "//button[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'Cerrar') or contains(., 'OK')]",
                "//a[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'Cerrar') or contains(., 'OK')]",
                "//div[contains(@class, 'close-button') or contains(@class, 'modal-close') or contains(@class, 'btn-close') or contains(@class, 'close-popup')]",
                "//span[contains(text(), 'x') or contains(text(), 'X') or @class='close-icon']",
                "//button[contains(@id, 'cookie') or contains(@id, 'modal') or contains(@class, 'cookie') or contains(@class, 'modal')][contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'OK')]",
                "//div[@role='dialog']//button[contains(., 'Aceptar') or contains(., 'Entendido') or contains(., 'OK')]"
            ]

            found_and_clicked = False
            for xpath_str in common_close_xpaths:
                try:
                    btn = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, xpath_str))
                    )
                    if btn.is_displayed() and btn.is_enabled():
                        btn.click()
                        time.sleep(2)
                        found_and_clicked = True
                        break
                except:
                    pass

        except Exception as e:
            pass # No imprimir errores de pop-up para mantener la salida limpia

        # --- ESPERAR A QUE EL CONTENIDO PRINCIPAL CARGUE ---
        try:
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'cont-package')))
        except Exception as e:
            # Intentar scroll y reintentar la espera si el elemento no es visible inicialmente
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)

            try:
                wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'cont-package')))
            except Exception as e:
                # Este es el único ERROR FATAL que se imprimirá si no se encuentra el elemento clave
                print(f"ERROR FATAL: 'cont-package' no se hizo visible incluso después de scroll. {e}")
                print("La estructura de la página o la forma en que carga el contenido ha cambiado significativamente o hay un bloqueo persistente.")
                return []

        time.sleep(5) # Tiempo adicional para asegurar la carga completa

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='cont-package')

        if not plan_elements:
            print("ERROR: Después de la carga, no se encontraron elementos con la clase 'cont-package' en BeautifulSoup.")
            print("Esto podría indicar que la clase ha cambiado o el contenido no está en el HTML parseado.")
            return []

        for i, plan_element in enumerate(plan_elements):
            nombre_plan = 'N/A'
            precio = 'N/A'
            gigas = 'N/A'
            apps_ilimitadas = 'No especificado'
            minutos_llamadas = 'No especificado'
            sms = 'No especificado'
            detalles_gigas = 'N/A' # Nuevo campo para detalles de gigas

            full_plan_text_raw = plan_element.get_text(separator=' ', strip=True)
            full_plan_text_lower = full_plan_text_raw.lower()

            # 2. Extraer Precio
            match_precio = re.search(r's/\s*(\d+\.\d+)', full_plan_text_lower)
            if match_precio:
                precio = f"S/ {match_precio.group(1)}"
            else:
                price_element_a = plan_element.find('a', href="javascript:void(0);")
                if price_element_a:
                    title_attr = price_element_a.get('title')
                    if title_attr:
                        match_price_title = re.search(r'(\d+\.\d+)', title_attr)
                        if match_price_title:
                            precio = f"S/ {match_price_title.group(1)}"

                if precio == 'N/A':
                    match_precio_simple = re.search(r'(\d+\.\d+)', full_plan_text_lower)
                    if match_precio_simple:
                        precio_val = float(match_precio_simple.group(1))
                        if 10.00 <= precio_val <= 200.00:
                            precio = f"S/ {match_precio_simple.group(1)}"

            # 1. Modificar Nombre del Plan para ser "Ilimitado - [Precio del Plan]"
            if precio != 'N/A':
                nombre_plan = f"Ilimitado - {precio.replace('S/ ', '')}"
            else:
                name_span_element = plan_element.find('span', class_='color-white text-bold title-1g')
                if name_span_element:
                    span_text = name_span_element.get_text(strip=True)
                    cleaned_span_text = re.sub(r'\s*S/\s*\d+\.\d+', '', span_text, flags=re.IGNORECASE).strip()
                    cleaned_span_text = re.sub(r'\s*\d+\.\d+$', '', cleaned_span_text, flags=re.IGNORECASE).strip()
                    if cleaned_span_text:
                        nombre_plan = f"Ilimitado - {cleaned_span_text}"
                    else:
                        nombre_plan = "Ilimitado - Precio No Disponible"
                else:
                    nombre_plan = "Ilimitado - Precio No Disponible"

            # 3. Extraer Gigas y Detalles de Gigas
            gigas_element = plan_element.find('p', class_='capa')

            if gigas_element:
                gigas_text = gigas_element.get_text(strip=True)
                gigas_text_lower = gigas_text.lower()

                period_element = plan_element.find('p', class_='period')
                period_text_lower = period_element.get_text(strip=True).lower() if period_element else ''

                if 'ilimitados' in gigas_text_lower:
                    gigas = 'Ilimitados'
                    match_gigas_alta_velocidad = re.search(r'(\d+)\s*GB(?:\s*en\s*alta\s*velocidad)?', gigas_text, re.IGNORECASE)
                    if match_gigas_alta_velocidad:
                        detalles_gigas = f"{match_gigas_alta_velocidad.group(1)} GB en Alta Velocidad"
                    elif 'alta velocidad' in period_text_lower:
                        detalles_gigas = "Velocidad reducida después de cierto consumo"
                    else:
                        detalles_gigas = "Datos ilimitados sin restricciones de velocidad explícitas"
                else:
                    match_gigas = re.search(r'(\d+)\s*GB', gigas_text, re.IGNORECASE)
                    if match_gigas:
                        gigas = f"{match_gigas.group(1)} GB"
                        if 'alta velocidad' in period_text_lower:
                            detalles_gigas = "En Alta Velocidad"
                    else:
                        gigas = gigas_text
                        if 'alta velocidad' in period_text_lower:
                            detalles_gigas = "En Alta Velocidad"
            else:
                gigas = 'N/A'
                detalles_gigas = 'Elemento de gigas no encontrado'


            # 4. Extraer Apps Ilimitadas
            app_keywords_patterns = {
                'WhatsApp': r'whatsapp ilimitado',
                'Facebook': r'facebook ilimitado',
                'Instagram': r'instagram ilimitado',
                'TikTok': r'tiktok ilimitado',
                'Spotify': r'spotify ilimitado',
                'Waze': r'waze ilimitado',
                'YouTube': r'youtube ilimitado',
                'Apps ilimitadas x meses': r'apps ilimitadas (x \d+ meses)?',
                'Internet + Llamadas ilimitadas': r'internet \+\s*llamadas ilimitadas'
            }
            found_apps = []
            for app_name, pattern in app_keywords_patterns.items():
                if re.search(pattern, full_plan_text_lower):
                    if 'x \d+ meses' in pattern and re.search(r'x \d+ meses', full_plan_text_lower):
                        promo_match = re.search(r'x \d+ meses', full_plan_text_lower).group(0)
                        found_apps.append(f"{app_name.replace(' x meses', '')} {promo_match}")
                    else:
                        found_apps.append(app_name)

            app_image_elements = plan_element.find_all('li', class_='app-item')
            for app_li in app_image_elements:
                img_tag = app_li.find('img')
                if img_tag and 'alt' in img_tag.attrs:
                    app_name_from_alt = img_tag['alt'].strip()
                    if app_name_from_alt and app_name_from_alt.lower() not in [app.lower() for app in found_apps]:
                        found_apps.append(app_name_from_alt)

            apps_ilimitadas = ", ".join(sorted(list(set(found_apps)))) if found_apps else 'No especificado'

            # Minutos/Llamadas y SMS
            minutos_llamadas = "No especificado"
            sms = "No especificado"

            todo_ilimitado_element = plan_element.find('p', class_=re.compile(r'title.*Todo ilimitado'))
            if todo_ilimitado_element and "todo ilimitado" in todo_ilimitado_element.get_text(strip=True).lower():
                minutos_llamadas = 'Llamadas ilimitadas'
                sms = 'SMS ilimitados'
            else:
                if 'llamadas ilimitadas perú' in full_plan_text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas Perú'
                    match_usa_canada = re.search(r'(\d+)\s*minutos\s*(?:para|a)\s*(?:usa|eeuu)\s*(?:y|e)\s*canadá', full_plan_text_lower)
                    if match_usa_canada:
                        minutos_llamadas += f", {match_usa_canada.group(1)} minutos para Usa y Canadá"
                elif 'llamadas ilimitadas' in full_plan_text_lower:
                    minutos_llamadas = 'Llamadas ilimitadas'

                if 'sms ilimitados' in full_plan_text_lower:
                    sms = 'SMS ilimitados'
                else:
                    match_sms = re.search(r'(\d+)\s*sms', full_plan_text_lower)
                    if match_sms:
                        sms = f"{match_sms.group(1)} SMS"

            benefit_period_element = plan_element.find('p', class_='benefit-period')
            if benefit_period_element and "internet, llamadas y sms" in benefit_period_element.get_text(strip=True).lower():
                if minutos_llamadas == "No especificado":
                    minutos_llamadas = 'Llamadas incluidas'
                if sms == "No especificado":
                    sms = 'SMS incluidos'

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Detalles de Gigas': detalles_gigas, # Añadir el nuevo campo
                'Apps Ilimitadas': apps_ilimitadas,
                'Minutos/Llamadas': minutos_llamadas,
                'SMS': sms
            })

    except Exception as e:
        print(f"Ocurrió un error crítico durante la ejecución: {e}")
        return []
    finally:
        if driver:
            driver.quit()

    return planes_data

if __name__ == "__main__":
    planes = extraer_planes_bitel_colab()
    if planes:
        df = pd.DataFrame(planes)

        # Ordenar por precio, asegurando que los valores 'N/A' vayan al final
        df['Precio_Sort'] = df['Precio (S/)'].apply(lambda x: float(str(x).replace('S/ ', '')) if isinstance(x, str) and 'S/' in x else (x if isinstance(x, (int, float)) else float('inf')))
        df_sorted = df.sort_values(by='Precio_Sort').drop(columns='Precio_Sort')

        # Estilos CSS personalizados para la tabla HTML (colores de Bitel)
        html_style = """
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f4f4f4; color: #333; }
            h1 { color: #007bff; text-align: center; margin-bottom: 20px; }
            table {
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
                box-shadow: 0 4px 12px rgba(0,0,0,0.15);
                background-color: #fff;
                border-radius: 8px;
                overflow: hidden;
            }
            th, td {
                padding: 15px 20px;
                text-align: left;
                border-bottom: 1px solid #eee;
            }
            th {
                background-color: #00AEEF; /* Azul Bitel */
                color: white;
                text-transform: uppercase;
                font-size: 0.95em;
                letter-spacing: 0.5px;
            }
            tr:nth-child(even) {
                background-color: #f8f8f8;
            }
            tr:hover {
                background-color: #e0f7fa; /* Un azul claro al pasar el mouse */
            }
            /* Estilo para la línea del encabezado */
            .header-line {
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                font-size: 1.2em;
                font-weight: bold;
                color: #555;
                text-align: center;
                margin-bottom: 25px;
                padding-bottom: 10px;
                border-bottom: 2px solid #ddd;
            }
        </style>
        """

        # Generar la tabla HTML desde el DataFrame ordenado
        html_table = df_sorted.to_html(index=False, escape=False, classes='bitel-plans-table')

        # Combinar todas las partes en un documento HTML completo
        full_html_output = f"""
        <!DOCTYPE html>
        <html lang="es">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Planes y Beneficios Bitel Perú</title>
            {html_style}
        </head>
        <body>
            <div class="header-line">--- Planes y Beneficios (ordenados por precio) ---</div>
            {html_table}
        </body>
        </html>
        """

        # Mostrar el HTML directamente en la celda de salida de Colab
        display(HTML(full_html_output))
    else:
        # Si no se extraen planes, el mensaje de error ya habrá sido impreso por la función.
        pass


Nombre del Plan,Precio (S/),Gigas,Detalles de Gigas,Apps Ilimitadas,Minutos/Llamadas,SMS
Ilimitado - 27.90,S/ 27.90,75 GB,En Alta Velocidad,"Apps ilimitadas x 12 meses, Facebook, Instagram, Paramount +, Spotify, TV360, Whatsapp",Llamadas incluidas,SMS incluidos
Ilimitado - 34.90,S/ 34.90,110 GB,En Alta Velocidad,"Paramount +, Spotify, TV360",Llamadas incluidas,SMS incluidos
Ilimitado - 39.90,S/ 39.90,125 GB,En Alta Velocidad,"Apps ilimitadas x 12 meses, Facebook, Instagram, Paramount +, Spotify, TV360",Llamadas incluidas,SMS incluidos
Ilimitado - 39.90,S/ 39.90,30 GB,En Alta Velocidad,"Apps ilimitadas x meses, Facebook, Instagram, Spotify, TV360, Whatsapp",Llamadas incluidas,SMS incluidos
Ilimitado - 46.10,S/ 46.10,100 GB,En Alta Velocidad,"Apps ilimitadas x 6 meses, Facebook, Instagram, Paramount +, Spotify, TV360",Llamadas incluidas,SMS incluidos
Ilimitado - 49.90,S/ 49.90,45 GB,En Alta Velocidad,"Apps ilimitadas x meses, Facebook, Instagram, Spotify, TV360, Whatsapp",Llamadas incluidas,SMS incluidos
Ilimitado - 52.90,S/ 52.90,160 GB,En Alta Velocidad,"Apps ilimitadas x 12 meses, Facebook, Instagram, Paramount +, Spotify, TV360",Llamadas incluidas,SMS incluidos
